In [1]:
%matplotlib inline

import glob
import matplotlib.pyplot as plt
import nilearn.image, nilearn.masking, nilearn.plotting
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

In [2]:
data_dir = '/Volumes/clmnlab/GA/MVPA/LSS_pb02_MO_short_duration/data/'

In [3]:
def fast_masking(img, roi):
    img_data = img.get_data()
    roi_mask = roi.get_data().astype(bool)
    
    if img_data.shape[:3] != roi_mask.shape:
        raise ValueError('different shape while masking! img=%s and roi=%s' % (img_data.shape, roi_mask.shape))
        
    return img_data[roi_mask, :].T

In [4]:
def accuracy(train_X, train_y, test_X, test_y):
    model.fit(train_X, train_y)
    return np.mean(model.predict(test_X) == test_y)

def cross_validation(datasets, target):
    train_y = target + target
    test_y = target
    
    result = []
    
    for i in range(len(datasets)):
        train_X = np.concatenate([a for j, a in enumerate(datasets) if j != i])
        test_X = datasets[i]
        
        result.append(accuracy(train_X, train_y, test_X, test_y))
        
    return np.mean(result)

In [7]:
roi_dir = '/Volumes/clmnlab/GA/fmri_data/masks/DMN/'

roi_path = sorted(glob.glob(roi_dir + '*.nii'))
#roi_path = [roi_dir + 'Core.nii', roi_dir + 'dMsub.nii',  roi_dir + 'MTLsub.nii']
roi_imgs = []

for path in roi_path:
    roi_imgs.append(nilearn.image.load_img(path))

In [8]:
roi_path

['/Volumes/clmnlab/GA/fmri_data/masks/DMN/Core.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/Core_PCC_l.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/Core_PCC_r.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/Core_aMPFC_l.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/Core_aMPFC_r.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/MTLsub.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/MTLsub_HF_l.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/MTLsub_HF_r.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/MTLsub_PHC_l.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/MTLsub_PHC_r.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/MTLsub_Rsp_l.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/MTLsub_Rsp_r.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/MTLsub_pIPL_l.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/MTLsub_pIPL_r.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/MTLsub_vMPFC.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/dMsub.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/DMN/dMsub_L

In [7]:
subj_list = [
    'GA01', 'GA02', 'GA05', 'GA07', 'GA08', 'GA11', 'GA12', 'GA13', 'GA14', 'GA15',
    'GA18', 'GA19', 'GA20', 'GA21', 'GA23', 'GA26', 'GA27', 'GA28', 'GA29', 'GA30',
    'GA31', 'GA32', 'GA33', 'GA34', 'GA35', 'GA36', 'GA37', 'GA38', 'GA42', 'GA44'
]
    
#    'GB01', 'GB02', 'GB05', 'GB07', 'GB08', 'GB11', 'GB12', 'GB13', 'GB14', 'GB15',
#    'GB18', 'GB19', 'GB20', 'GB21', 'GB23', 'GB26', 'GB27', 'GB28', 'GB29', 'GB30',
#    'GB31', 'GB32', 'GB33', 'GB34', 'GB35', 'GB36', 'GB37', 'GB38', 'GB42', 'GB44'
#]

In [ ]:
data = {}

for subj in subj_list:
    for run in range(1, 7):
        data[subj, run] = nilearn.image.load_img(data_dir + 'betasLSS.MO.shortdur.%s.r%02d.nii.gz' % (subj, run))
    print(subj, end='\r')

In [ ]:
for key, value in data.items():
    data[key] = nilearn.image.index_img(value, np.arange(1, 97))

In [ ]:
target_path = list(range(1, 13)) * 8
target_pos = []
 
with open('/Volumes/clmnlab/GA/MVPA/LSS_pb02/behaviors/targetID.txt') as file:
    for line in file:
        target_pos.append(int(line.strip()))
        
target_pos = target_pos[1:97]

In [ ]:
model = SVC()

results = {}
 
for subj in subj_list:
    for m, roi in enumerate(roi_imgs):
        roi_index = m+1
        
        if ('pos', 'unpracticed', subj, roi_index) in results:
            continue
        
        X = [
            fast_masking(roi=roi, img=data[subj, 1]), 
            fast_masking(roi=roi, img=data[subj, 2]), 
            fast_masking(roi=roi, img=data[subj, 3]), 
        ]
        
        results['pos', 'practiced', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'practiced', subj, roi_index] = cross_validation(X, target_path)
        
        X = [
            fast_masking(roi=roi, img=data[subj, 4]), 
            fast_masking(roi=roi, img=data[subj, 5]), 
            fast_masking(roi=roi, img=data[subj, 6]), 
        ]
        
        results['pos', 'unpracticed', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'unpracticed', subj, roi_index] = cross_validation(X, target_path)
        
        print(subj, roi_index, end='\r')

In [ ]:
len(results)

In [ ]:
results

In [ ]:
data = []

In [ ]:
subj_list = [
#    'GA01', 'GA02', 'GA05', 'GA07', 'GA08', 'GA11', 'GA12', 'GA13', 'GA14', 'GA15',
#    'GA18', 'GA19', 'GA20', 'GA21', 'GA23', 'GA26', 'GA27', 'GA28', 'GA29', 'GA30',
#    'GA31', 'GA32', 'GA33', 'GA34', 'GA35', 'GA36', 'GA37', 'GA38', 'GA42', 'GA44'
#]
    
   'GB01', 'GB02', 'GB05', 'GB07', 'GB08', 'GB11', 'GB12', 'GB13', 'GB14', 'GB15',
   'GB18', 'GB19', 'GB20', 'GB21', 'GB23', 'GB26', 'GB27', 'GB28', 'GB29', 'GB30',
   'GB31', 'GB32', 'GB33', 'GB34', 'GB35', 'GB36', 'GB37', 'GB38', 'GB42', 'GB44'
]

data = {}

for subj in subj_list:
    for run in range(1, 7):
        data[subj, run] = nilearn.image.load_img(data_dir + 'betasLSS.MO.shortdur.%s.r%02d.nii.gz' % (subj, run))
    print(subj, end='\r')

In [ ]:
for key, value in data.items():
    data[key] = nilearn.image.index_img(value, np.arange(1, 97))

In [ ]:
target_path = list(range(1, 13)) * 8
target_pos = []

with open('/Volumes/clmnlab/GA/MVPA/LSS_pb02/behaviors/targetID.txt') as file:
    for line in file:
        target_pos.append(int(line.strip()))
        
target_pos = target_pos[1:97]

In [ ]:
for subj in subj_list:
    for m, roi in enumerate(roi_imgs):
        roi_index = m+1
        
        if ('pos', 'unpracticed', subj, roi_index) in results:
            continue
        
        X = [
            fast_masking(roi=roi, img=data[subj, 1]), 
            fast_masking(roi=roi, img=data[subj, 2]), 
            fast_masking(roi=roi, img=data[subj, 3]), 
        ]
        
        results['pos', 'practiced', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'practiced', subj, roi_index] = cross_validation(X, target_path)
        
        X = [
            fast_masking(roi=roi, img=data[subj, 4]), 
            fast_masking(roi=roi, img=data[subj, 5]), 
            fast_masking(roi=roi, img=data[subj, 6]), 
        ]
        
        results['pos', 'unpracticed', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'unpracticed', subj, roi_index] = cross_validation(X, target_path)
        
        print(subj, roi_index, end='\r')

In [ ]:
df = pd.DataFrame(columns=['target', 'practiced', 'subject', 'roi', 'accu'])
for i,j in zip(results.keys(), results.values()):
    df = df.append({'target':i[0], 'practiced':i[1], 'subject':i[2], 'roi':i[3], 'accu':j}, ignore_index=True)
    
df.head()

In [ ]:
df.to_csv("20190906_DMN_svc.csv")